In [1]:
import pandas as pd

In [2]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

# Data Preview

In [3]:
data_coding.head()


,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,...,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,0,Abi Bell,Liberal Democrats,1936.07,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Abigail Baker,Liberal Democrats,748.80,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Adam Prices,Plaid Cymru,275.85,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adam Williams,Liberal Democrats,484.06,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Addison Lee,Conservative Party,22648.35,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data_invoice.head()

,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,...,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
0,SP0555253,Plaid Cymru,UK Parliamentary general election 12/12/2019,Political Party,200.40,03/01/2019,Unsolicited material to electors,Low Cost Printing,"Precision House, Pentrpoeth Rd, Swansea, SA6 6...",£0.00,...,£0.00,NaN,03/01/2019,74031,77,2276,NaN,NaN,NaN,False
1,SP0541022,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,224.12,19/02/2019,Unsolicited material to electors,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United ...",£224.12,...,£0.00,19/02/2019,25/02/2019,71688,90,76765,NaN,NaN,NaN,False
2,SP0541021,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,498.06,15/04/2019,Unsolicited material to electors,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United ...",£498.06,...,£0.00,15/04/2019,08/05/2019,71687,90,76765,NaN,NaN,NaN,False
3,SP0541019,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,239.40,05/11/2018,Unsolicited material to electors,Letter Workshop t/a Election Workshop,"Unit 3, Sheffield Street, MANCHESTER, MANCHEST...",£239.40,...,£0.00,05/11/2018,05/11/2018,71686,90,68079,NaN,NaN,NaN,False
4,SP0541018,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,202.50,19/02/2019,Unsolicited material to electors,Chelmsford Liberal Independent Print Society,"61 Galleywood Road, Chelmsford, CM2 8BY, Unite...",£202.50,...,£0.00,19/02/2019,12/03/2019,71685,90,76746,NaN,NaN,NaN,False


# Test

In [5]:
def get_accounted_output(name,accounted,unaccounted,overaccounted):
    acc_total = sum([accounted,unaccounted,overaccounted])
    return '{:<55s}'.format(name+':') \
            + '{:<40s}'.format('accounted = £'+'{0:.2f}'.format(accounted)+' ('+'{0:.1f}'.format((100*accounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| un-accounted = £'+'{0:.2f}'.format(unaccounted)+' ('+'{0:.1f}'.format((100*unaccounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| over-accounted = £'+'{0:.2f}'.format(overaccounted)+' ('+'{0:.1f}'.format((100*overaccounted/acc_total) if acc_total>0 else 0)+'%)')+'\n'


output=''
parties = data_coding['Party'].unique()
categories = data_coding['Expense Category'].unique()

total_accounted = 0
total_unaccounted = 0
total_overaccounted = 0

for party in parties:
    output += '\n####################################################################################\n  '+party+'\n####################################################################################\n\n'
    sub_party_coding = data_coding[ data_coding['Party']==party ]
    sub_party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    party_accounted = 0
    party_unaccounted = 0
    party_overaccounted = 0
    
    for category in categories:
        sub_cat_coding = sub_party_coding[ sub_party_coding['Expense Category']==category ]
        sub_cat_invoice = sub_party_invoice[ sub_party_invoice['ExpenseCategoryName']==category ]
        
        accounted = 0
        unaccounted = 0
        overaccounted = 0
        
        for idx, row in sub_cat_coding.iterrows():
            supplier = row['Supplier']
            spent = row['Total Spend']
            
            sub_pos_invoice = sub_cat_invoice[ sub_cat_invoice['SupplierName']==supplier ]
            filed = sub_pos_invoice['TotalExpenditure'].sum()

            accounted += min(spent,filed)
            unaccounted += max(spent-filed,0)
            overaccounted += max(filed-spent,0)
        
        party_accounted += accounted
        party_unaccounted += unaccounted
        party_overaccounted += overaccounted
    
        output += get_accounted_output(' Category '+category,accounted,unaccounted,overaccounted)
    output += '-'*170+'\n'
    output += get_accounted_output(' Total',party_accounted,party_unaccounted,party_overaccounted)
    
    total_accounted += party_accounted
    total_unaccounted += party_unaccounted
    total_overaccounted += party_overaccounted
    
print(get_accounted_output('Overall',total_accounted,total_unaccounted,total_overaccounted))
print(output)

Overall:                                               accounted = £31023496.55 (62.2%)        | un-accounted = £18537259.84 (37.2%)   | over-accounted = £283105.92 (0.6%)    


####################################################################################
  Liberal Democrats
####################################################################################

 Category Transport:                                   accounted = £194457.11 (72.1%)          | un-accounted = £71808.11 (26.6%)      | over-accounted = £3290.45 (1.2%)      
 Category Advertising:                                 accounted = £2532566.84 (29.6%)         | un-accounted = £6027861.01 (70.4%)    | over-accounted = £0.00 (0.0%)         
 Category Unsolicited material to electors:            accounted = £3506353.87 (98.7%)         | un-accounted = £47399.73 (1.3%)       | over-accounted = £0.00 (0.0%)         
 Category Campaign broadcasts:                         accounted = £71554.80 (91.4%)           | un-acc